In [2]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [3]:
CLIENT_ID = 'JRS51VSOAODS530A5VGF0QZRC4G5HM1R3J4U5KORQ2J3JJC0' # your Foursquare ID
CLIENT_SECRET = 'E1BBHI1SOFKFNKDCJ4MVLE5LNUN3WA2VTJMIMCSW0LGALL5T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JRS51VSOAODS530A5VGF0QZRC4G5HM1R3J4U5KORQ2J3JJC0
CLIENT_SECRET:E1BBHI1SOFKFNKDCJ4MVLE5LNUN3WA2VTJMIMCSW0LGALL5T


In [4]:
address = 'Bangalore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manhattan are 12.9791198, 77.5912997.


In [5]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByDistance=True'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JRS51VSOAODS530A5VGF0QZRC4G5HM1R3J4U5KORQ2J3JJC0&client_secret=E1BBHI1SOFKFNKDCJ4MVLE5LNUN3WA2VTJMIMCSW0LGALL5T&v=20180605&ll=12.9791198,77.5912997&radius=1000&limit=100&sortByDistance=True'

In [6]:
results = requests.get(url).json()

# define the dataframe columns
column_names = ['Name', 'Latitude', 'Longitude', 'Distance', 'CategoryName'] 

# instantiate the dataframe
outcome = pd.DataFrame(columns=column_names)

venues = results['response']['groups'][0]['items']
for venue in venues:
    name = venue['venue']['name'] #name
    lat = venue['venue']['location']['lat'] #latitude
    long = venue['venue']['location']['lng'] #lng
    distance = venue['venue']['location']['distance'] # distance
    categoryname = venue['venue']['categories'][0]['name'] #category name
    outcome = outcome.append({'Name': name,
                              'Latitude': lat,
                              'Longitude': long,
                              'Distance': distance,
                              'CategoryName': categoryname}, ignore_index=True)
outcome.head()

,Name,Latitude,Longitude,Distance,CategoryName
0,Vidhana Soudha,12.979027,77.591881,63,Capitol Building
1,Cubbon Park,12.977042,77.595277,489,Park
2,KSLTA Tennis stadium,12.975094,77.594439,562,Tennis Stadium
3,Nisarga Garden Restaurant,12.983332,77.594431,578,Indian Restaurant
4,Atria Hotel,12.980227,77.586067,580,Hotel


In [7]:
#create map of manhattan with markers for venues
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, categoryname in zip(outcome['Latitude'], outcome['Longitude'], outcome['CategoryName']):
    label = '{}'.format(categoryname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

In [8]:
categorynamedf = outcome[['CategoryName']]
categorynamedf['Number of Occurences'] = 1
categorynamedf = categorynamedf.groupby('CategoryName').count()
categorynamedf['Percentage of Occurences'] = categorynamedf['Number of Occurences']/(categorynamedf['Number of Occurences'].sum())
categorynamedf = categorynamedf.sort_values(by='Percentage of Occurences', ascending=False)
categorynamedf

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Number of Occurences,Percentage of Occurences
CategoryName,,
Indian Restaurant,9,0.123288
Hotel,6,0.082192
Café,4,0.054795
Lounge,4,0.054795
Italian Restaurant,4,0.054795
Coffee Shop,4,0.054795
Mexican Restaurant,3,0.041096
Pizza Place,2,0.027397
Asian Restaurant,2,0.027397
